Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give read access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [ ]:
## DONT CHANGE THIS CELL
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip

--2021-10-11 13:08:41--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip [following]
--2021-10-11 13:08:42--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217313 (212K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 212.22K   101KB/s    in 2.1s    

2021-10-11 13:08:45 (101 KB/s) - ‘data.zip’ saved [217313/217313]



In [ ]:
## Import relevant packages

import os

import time
import datetime

import torch
import torch.optim as O
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

import logging
from argparse import ArgumentParser

from pdb import set_trace

In [ ]:
import zipfile
from torchtext.legacy.data import Field, BucketIterator, TabularDataset
import spacy
import pandas as pd
import numpy as np
import random

In [ ]:
torch.manual_seed(500)
torch.cuda.manual_seed(500)
np.random.seed(500)
random.seed(500)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.deterministic = False

In [ ]:
with zipfile.ZipFile("./data.zip", 'r') as zip_ref:
    zip_ref.extractall("./NLP_A3")

In [ ]:
training_data_path = "./NLP_A3/data/train/train.data.txt"
validation_data_path = "./NLP_A3/data/validation/validation.data.txt"
training_data_predict_path = "./NLP_A3/data/train/train.gold.txt"
validation_data_predict_path = "./NLP_A3/data/validation/validation.gold.txt"

In [ ]:
"""checking arguments"""
def check_args(args):
	# --result_dir
	check_folder(os.path.join(args['results_dir'], args['model'], args['dataset']))

	# --epoch
	try:
			assert args['epochs'] >= 1
	except:
			print('number of epochs must be larger than or equal to one')

	# --batch_size
	try:
			assert args['batch_size'] >= 1
	except:
			print('batch size must be larger than or equal to one')
	return args

def get_device(gpu_no):
	if torch.cuda.is_available():
		torch.cuda.set_device(gpu_no)
		return torch.device('cuda:{}'.format(gpu_no))
	else:
		return torch.device('cpu')

def makedirs(name):
	"""helper function for python 2 and 3 to call os.makedirs()
		avoiding an error if the directory to be created already exists"""

	import os, errno

	try:
		os.makedirs(name)
	except OSError as ex:
		if ex.errno == errno.EEXIST and os.path.isdir(name):
			# ignore existing directory
			pass
		else:
			# a different error happened
			raise

def check_folder(log_dir):
	if not os.path.exists(log_dir):
		os.makedirs(log_dir)
	return log_dir

def get_logger(args, phase):
	logging.basicConfig(level=logging.INFO, 
												filename = "{}/{}/{}/{}.log".format(args['results_dir'], args['model'], args['dataset'], phase),
												format = '%(asctime)s - %(message)s', 
												datefmt='%d-%b-%y %H:%M:%S')
	return logging.getLogger(phase)

In [ ]:
def Dataset(batch_size):
  file_ = pd.read_csv(training_data_path,sep='\t', header = None, names = ['word', 'POS', 'index_sent1', 'sent1', 'sent2'])
  file_v = pd.read_csv(validation_data_path,sep='\t', header = None, names = ['word', 'POS', 'index_sent1', 'sent1', 'sent2'])
  file_['index_sent2'] = file_['index_sent1']
  file_v['index_sent2'] = file_v['index_sent1']
  for i in file_.index:
    idx1 = file_.at[i, 'index_sent1']
    file_.at[i, 'index_sent1'] = idx1[0]
    idx2 = file_.at[i, 'index_sent2']
    file_.at[i, 'index_sent2'] = idx2[-1]
  for i in file_v.index:
    idx1 = file_v.at[i, 'index_sent1']
    file_v.at[i, 'index_sent1'] = idx1[0]
    idx2 = file_v.at[i, 'index_sent2']
    file_v.at[i, 'index_sent2'] = idx2[-1]
  with open(training_data_predict_path) as f:
    data_predict = f.readlines()
    f.close()
  with open(validation_data_predict_path) as f:
    data_predict_v = f.readlines()
    f.close()
  file_predict = pd.DataFrame(data_predict, columns= ['category'])
  file_predict_v = pd.DataFrame(data_predict_v, columns= ['category'])
  for i in file_predict.index:
    idx1 = file_predict.at[i, 'category']
    if idx1[0] == 'T':
      file_predict.at[i, 'category'] = 1
    else:
      file_predict.at[i, 'category'] = 0
  for i in file_predict_v.index:
    idx1 = file_predict_v.at[i, 'category']
    if idx1[0] == 'T':
      file_predict_v.at[i, 'category'] = 1
    else:
      file_predict_v.at[i, 'category'] = 0
  file_['category'] = file_predict['category']
  file_v['category'] = file_predict_v['category']
  for i in file_.index:
    idx1 = file_.at[i, 'POS']
    if idx1[0] == 'N':
      file_.at[i, 'POS'] = 1
    else:
      file_.at[i, 'POS'] = 0
  for i in file_v.index:
    idx1 = file_v.at[i, 'POS']
    if idx1[0] == 'N':
      file_v.at[i, 'POS'] = 1
    else:
      file_v.at[i, 'POS'] = 0
  file_.to_json('train_pos.json', orient = 'records', lines = True)
  file_v.to_json('valid_pos.json', orient = 'records', lines = True)
  load_model_lemma = spacy.load('en', disable = ['parser','ner'])
  def tokenize_lemma(text):
    doc = load_model_lemma(text)
    return [token.lemma_ for token in doc]

  idx_sent = Field(sequential=False, use_vocab=False)
  sent = Field(sequential=True, use_vocab=True, tokenize=tokenize_lemma, lower=True)
  category = Field(sequential=False, use_vocab=False)
  fields = {'word': ('w', sent), "POS":("pos", category), "index_sent1": ('idx1', idx_sent), "index_sent2": ('idx2', idx_sent), "sent1":('s1', sent), "sent2":('s2',sent), "category":("cate", category)}
  train_data, valid_data = TabularDataset.splits(path="", train="train_pos.json", validation="valid_pos.json", format="json", fields=fields)
  sent.build_vocab(train_data,vectors="glove.6B.300d")
  train_iterator, valid_iterator = BucketIterator.splits((train_data,valid_data), batch_size=batch_size, device="cuda", sort_key = lambda x: len(x.s1), sort_within_batch=True)
  return train_iterator, valid_iterator, sent

In [ ]:
def adjust_dim(word_idx, hidden_size):
  word_idx = torch.unsqueeze(word_idx, 0)
  word_idx = torch.unsqueeze(word_idx, 2)
  word_idx = torch.repeat_interleave(word_idx, hidden_size, dim=2)
  return word_idx

In [ ]:
class Context_word_embedding_model(nn.Module):
  def __init__(self, input_size, embed_size, hidden_size, num_layers, bidirectional, num_direction, dropout, device):
    super(Context_word_embedding_model, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.num_direction = num_direction
    self.embedding = nn.Embedding(input_size, embed_size,padding_idx= 1)
    self.dimension = hidden_size*num_direction
    self.device = device
    self.drop = nn.Dropout(p =0.1)
    self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, bidirectional = bidirectional, dropout = dropout)
    self.fc = nn.Linear(self.dimension, self.dimension)
    self.fc_pos = nn.Linear(self.dimension, 1)
    self.cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    self.sig = nn.Sigmoid()

  def forward(self, batch):
    x = batch.s1.to(device=self.device)
    s2 = batch.s2.to(device=self.device)
    word_idx = batch.idx1.to(device=self.device)
    word_idx2 = batch.idx2.to(device=self.device)
    h0 = torch.zeros(self.num_layers*self.num_direction, x.size(1), self.hidden_size).to(self.device)
    c0 = torch.zeros(self.num_layers*self.num_direction, x.size(1), self.hidden_size).to(self.device)
    h02 = torch.zeros(self.num_layers*self.num_direction, s2.size(1), self.hidden_size).to(self.device)
    c02 = torch.zeros(self.num_layers*self.num_direction, s2.size(1), self.hidden_size).to(self.device)

    embedded_ = self.embedding(x)
    embedded2_ = self.embedding(s2)
    embedded = self.drop(embedded_)
    embedded2 = self.drop(embedded2_)
    outputs, _ = self.lstm(embedded, (h0,c0))
    outputs2, __ = self.lstm(embedded2, (h02,c02))
    word_idx = adjust_dim(word_idx, self.hidden_size*self.num_direction)
    word_idx2 = adjust_dim(word_idx2, self.hidden_size*self.num_direction)
    embed = torch.gather(outputs, 0, word_idx).squeeze()
    embed2 = torch.gather(outputs2, 0, word_idx2).squeeze()
    context_embedding = self.fc(embed)
    context_embedding2 = self.fc(embed2)
    out_sim = self.cos(context_embedding, context_embedding2)
    out_final = self.sig(out_sim)
    return out_final


In [ ]:
arg_dictionary = {}

In [ ]:
arg_dictionary['gpu'] = 0
arg_dictionary['batch_size'] = 128
arg_dictionary['embed_dim'] = 300
arg_dictionary['d_hidden'] = 200
arg_dictionary['dp_ratio'] = 0.2
arg_dictionary['epochs'] = 20
arg_dictionary['lr'] = 0.001
arg_dictionary['combine'] = 'cat'
arg_dictionary['results_dir'] = 'results'
arg_dictionary['dataset'] = 'mnli'
arg_dictionary['model'] = 'lstm'

In [ ]:
arg_dictionary['results_dir']

'results'

In [ ]:
## Basic training loop

class Train():
	def __init__(self):
		print("program execution start: {}".format(datetime.datetime.now()))
		self.args = check_args(arg_dictionary)
		self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
		self.logger = get_logger(self.args, "train")
		self.logger.info("Arguments: {}".format(self.args))
		
		dataset_options = {
											'batch_size': self.args['batch_size'], 
											'device': self.device
										}

    ## TODO: Load your own dataset
		self.train_iter, self.valid_iter, self.sent = Dataset(dataset_options['batch_size'])
		self.input_size = len(self.sent.vocab)
		self.embedding_size = 300
		self.hidden_size = 200
		self.num_layers = 1
		self.bidirectional = False
		self.num_direction = 1
		self.dropout = 0
    ## TODO: Load your own model
		self.model = Context_word_embedding_model(self.input_size, self.embedding_size, self.hidden_size, self.num_layers, self.bidirectional, self.num_direction, self.dropout, self.device)
		self.pretrained_embeddings = self.sent.vocab.vectors
		self.model.embedding.weight.data.copy_(self.pretrained_embeddings)
		self.model.to(self.device)
		self.criterion = nn.BCELoss()
		self.opt = O.Adam(self.model.parameters(), lr = self.args['lr'],weight_decay=2e-4)
		self.best_val_acc = 57
		self.scheduler = StepLR(self.opt, step_size=5, gamma=0.5)

		print("resource preparation done: {}".format(datetime.datetime.now()))

	def result_checkpoint(self, epoch, train_loss, val_loss, train_acc, val_acc, took):
		if self.best_val_acc is None or val_acc > self.best_val_acc:
			self.best_val_acc = val_acc
			torch.save({
				'accuracy': self.best_val_acc,
				# 'options': self.model_options,
				'model_dict': self.model.state_dict(),
			}, '{}/{}/{}/best-{}-{}-params.pt'.format(self.args['results_dir'], self.args['model'], self.args['dataset'], self.args['model'], self.args['dataset']))
		self.logger.info('| Epoch {:3d} | train loss {:5.2f} | train acc {:5.2f} | val loss {:5.2f} | val acc {:5.2f} | time: {:5.2f}s |'
				.format(epoch, train_loss, train_acc, val_loss, val_acc, took))
	
	def train(self):
		self.model.train(); self.train_iter.init_epoch()
		n_correct, n_total, n_loss = 0, 0, 0
		for batch_idx, batch in enumerate(self.train_iter):
			answer = self.model(batch)
			targets = batch.cate.to(device=self.device)
			loss = self.criterion(answer, targets.type_as(answer));x = answer > 0.5; predictions = list(map(int,x)); predictions = torch.tensor(predictions).to(self.device)
			n_correct += (predictions == targets).sum()
			self.opt.zero_grad()
			n_total += targets.shape[0]
			n_loss += loss.item()
			
			loss.backward(); self.opt.step()
		train_loss = n_loss/n_total
		train_acc = 100. * n_correct/n_total
		return train_loss, train_acc

	def validate(self):
		self.model.eval(); self.valid_iter.init_epoch()
		n_correct, n_total, n_loss = 0, 0, 0
		with torch.no_grad():
			for batch_idx, batch in enumerate(self.valid_iter):
				answer = self.model(batch)
				targets = batch.cate.to(device=self.device)
				loss = self.criterion(answer, targets.type_as(answer)); x = answer > 0.5 ;predictions = list(map(int,x));predictions = torch.tensor(predictions).to(self.device)
				n_correct += (predictions == targets).sum()
				n_total += targets.shape[0]
				n_loss += loss.item()

			val_loss = n_loss/n_total
			val_acc = 100. * n_correct/n_total
			return val_loss, val_acc

	def execute(self):
		print(" [*] Training starts!")
		print('-' * 99)
		for epoch in range(1, self.args['epochs']+1):
			start = time.time()

			train_loss, train_acc = self.train()
			val_loss, val_acc = self.validate()
			#self.scheduler.step()
			
			took = time.time()-start
			self.result_checkpoint(epoch, train_loss, val_loss, train_acc, val_acc, took)

			print('| Epoch {:3d} | train loss {:5.2f} | train acc {:5.2f} | val loss {:5.2f} | val acc {:5.2f} | time: {:5.2f}s |'.format(
				epoch, train_loss, train_acc, val_loss, val_acc, took))
		self.finish()

	def finish(self):
		self.logger.info("[*] Training finished!\n\n")
		print('-' * 99)
		print(" [*] Training finished!")
		print(" [*] Please find the saved model and training log in results_dir")


In [ ]:
## Start training
task = Train()


program execution start: 2021-10-11 13:08:47.393104
resource preparation done: 2021-10-11 13:09:42.422902


In [ ]:
task.execute()

 [*] Training starts!
---------------------------------------------------------------------------------------------------
| Epoch   1 | train loss  0.01 | train acc 53.50 | val loss  0.01 | val acc 56.74 | time:  1.10s |
| Epoch   2 | train loss  0.01 | train acc 60.34 | val loss  0.01 | val acc 55.02 | time:  0.99s |
| Epoch   3 | train loss  0.01 | train acc 63.74 | val loss  0.01 | val acc 58.78 | time:  0.98s |
| Epoch   4 | train loss  0.00 | train acc 67.83 | val loss  0.01 | val acc 59.09 | time:  0.99s |
| Epoch   5 | train loss  0.00 | train acc 70.50 | val loss  0.01 | val acc 60.03 | time:  0.97s |
| Epoch   6 | train loss  0.00 | train acc 73.54 | val loss  0.01 | val acc 58.93 | time:  0.98s |
| Epoch   7 | train loss  0.00 | train acc 75.44 | val loss  0.01 | val acc 59.87 | time:  0.96s |
| Epoch   8 | train loss  0.00 | train acc 77.43 | val loss  0.01 | val acc 58.62 | time:  0.96s |
| Epoch   9 | train loss  0.00 | train acc 77.93 | val loss  0.01 | val acc 58.15 | ti

In [ ]:
%ls

data.zip  NLP_A3/  results/  train_pos.json  valid_pos.json


In [ ]:
model_file_path = './results/lstm/mnli'

In [ ]:
task.sent

In [ ]:
import dill

In [ ]:
with open(os.path.join(model_file_path,'vocab'), 'wb') as file:
  dill.dump(task.sent, file)

In [ ]:
## Zip the final model and all the required files, such as vocabulary
# Replace USERID with your own, such as 2017CSZ8058
!zip -r 2018EE10493_A_model.zip **

## Upload it to Google drive and ensure that the testing notebook uses the correct link

  adding: data.zip (stored 0%)
  adding: NLP_A3/ (stored 0%)
  adding: NLP_A3/data/ (stored 0%)
  adding: NLP_A3/data/train/ (stored 0%)
  adding: NLP_A3/data/train/train.gold.txt (deflated 90%)
  adding: NLP_A3/data/train/train.data.txt (deflated 63%)
  adding: NLP_A3/data/validation/ (stored 0%)
  adding: NLP_A3/data/validation/validation.data.txt (deflated 58%)
  adding: NLP_A3/data/validation/validation.gold.txt (deflated 87%)
  adding: results/ (stored 0%)
  adding: results/lstm/ (stored 0%)
  adding: results/lstm/mnli/ (stored 0%)
  adding: results/lstm/mnli/best-lstm-mnli-params.pt (deflated 6%)
  adding: results/lstm/mnli/train.log (deflated 78%)
  adding: results/lstm/mnli/vocab (deflated 86%)
  adding: train_pos.json (deflated 76%)
  adding: valid_pos.json (deflated 74%)
